In [ ]:
import numpy as np
import cvxpy as cp
import scipy as sc
import pennylane as qml
import torch
import torch.nn as nn
import torch.nn.functional as Fn
import torch.optim as optim
from torch.autograd import Variable

In [ ]:
import torch
import torch.nn as nn

class neural_function(nn.Module):
    def __init__(self, dimension, hidden_layers):
        super(neural_function, self).__init__()

        self.dimension = dimension
        self.hidden_layers = hidden_layers

        # Create a list to hold the hidden layer modules
        self.hidden_layer_modules = nn.ModuleList()

        # Add the input layer
        self.hidden_layer_modules.append(nn.Linear(dimension, hidden_layers[0]))

        # Add the hidden layers
        for i in range(1, len(hidden_layers)):
            self.hidden_layer_modules.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))

        # Add the output layer
        self.lin_end = nn.Linear(hidden_layers[-1], 1)

    def forward(self, input):
        y = input.float()

        # Forward pass through each hidden layer
        for layer in self.hidden_layer_modules:
            y = torch.sigmoid(layer(y))

        # Forward pass through the output layer
        y = self.lin_end(y)

        return y

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class neural_function(nn.Module):
    def __init__(self, dimension, hidden_layers):
        super(neural_function, self).__init__()
        self.dimension = dimension
        self.hidden_layers = hidden_layers

        # Create hidden layers
        self.hidden_layer_modules = nn.ModuleList()
        self.hidden_layer_modules.append(nn.Linear(dimension, hidden_layers[0]))  
        for i in range(1, len(hidden_layers)):
            self.hidden_layer_modules.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))

        # Output layer 
        self.lin_end = nn.Linear(hidden_layers[-1], 1)

    def forward(self, input):
        y = input.float()
        
        for layer in self.hidden_layer_modules:
            y = F.relu(layer(y))  

        y = self.lin_end(y)
        
        return F.softplus(y) # Apply softplus to output

In [ ]:
import numpy as np

def generate_diagonal_positive_definite_matrices(dim):
    while True:
        # Generate random positive values for the diagonal entries
        diagonal_values_1 = np.random.rand(dim)
        diagonal_values_2 = np.random.rand(dim)

        # Construct the diagonal matrices A and B
        A = np.diag(diagonal_values_1)
        B = np.diag(diagonal_values_2)

        # Compute the trace of A
        trace_A = np.trace(A)
        trace_B = np.trace(B)

        # Normalize the matrices to have trace 1
        A /= trace_A
        B /= trace_B

        # Check if matrices A and B commute
        commutation_check = np.dot(A, B) - np.dot(B, A)

        if np.allclose(commutation_check, np.zeros((dim, dim))):
            return A, B

# Generate diagonal positive definite matrices A and B with trace 1 and they commute
N = 64
dim = N # Dimension of the matrices
A, B = generate_diagonal_positive_definite_matrices(dim)

# Print matrices A and B
print("Matrix A:")
print(A)
print("\nMatrix B:")
print(B)

In [ ]:
def check_if_commuting(rho, sigma):
    return np.matmul(rho, sigma) - np.matmul(sigma, rho)

In [ ]:
# quantum circuit settings
num_wires = 6
num_layers = 1
num_shots = 1
num_of_samples = 100

# initiate the quantum device
device = qml.device("default.mixed", wires=num_wires, shots=num_of_samples)

@qml.qnode(device)
def measure_rho(param):
    qml.QubitDensityMatrix(A, wires=np.arange(num_wires))
    #return qml.density_matrix(wires=np.arange(num_wires))
    qml.RandomLayers(param, wires=np.arange(num_wires), seed=seed_no)
    # measure in the computational basis
    result = qml.sample(qml.PauliZ(0)), qml.sample(qml.PauliZ(1)), qml.sample(qml.PauliZ(2)), qml.sample(qml.PauliZ(3)), qml.sample(qml.PauliZ(4)), qml.sample(qml.PauliZ(5))
    return result


@qml.qnode(device)
def measure_sigma(param):
    qml.QubitDensityMatrix(B, wires=np.arange(num_wires))
    #return qml.density_matrix(wires=np.arange(num_wires))
    qml.RandomLayers(param, wires=np.arange(num_wires), seed=seed_no)
    # measure in the computational basis
    result = qml.sample(qml.PauliZ(0)), qml.sample(qml.PauliZ(1)), qml.sample(qml.PauliZ(2)), qml.sample(qml.PauliZ(3)), qml.sample(qml.PauliZ(4)), qml.sample(qml.PauliZ(5))
    return result


In [ ]:
params_ran = np.random.random(qml.RandomLayers.shape(n_layers=num_layers, n_rotations=3))
rho_test = measure_rho(params_ran)
sigma_test = measure_sigma(params_ran)
print(rho_test)
print(sigma_test)

In [ ]:
#@title Optimization using Gradient Descent with neural network (softplus version)

# parameters of the optimization
num_of_epochs = 2500
learning_rate = 0.8
num_of_samples = 100
dimension = num_wires
hidden_layer = [100]
alpha = 1.05
# seed_no = 458


# initialize the neural network and quantum circuit parameters
neural_fn = neural_function(dimension, hidden_layer)
param_init = np.random.random(qml.RandomLayers.shape(n_layers=num_layers, n_rotations=3))


# intialize the cost function store
cost_func_store = []


# start the training
for epoch in range(1, num_of_epochs):


  # evaluate the gradient with respect to the quantum circuit parameters
    gradients = np.zeros_like((param_init))
    for i in range(len(gradients)):
        for j in range(len(gradients[0])):

      # copy the parameters
            shifted = param_init.copy()

      # right shift the parameters
            shifted[i, j] += np.pi/2

      # parameter-shift for the first term

      # forward evaluation
            forward_sum_1 = 0
            result = measure_rho(shifted)
            for sample in range(num_of_samples):
                sample_result_array = np.array([result[i][sample] for i in range(int(num_wires))])
                nn_result = neural_fn(torch.from_numpy(sample_result_array))
                forward_sum_1 += (nn_result[0].detach().numpy())**((alpha-1)/alpha)

      # normalize this sum
            forward_sum_1 = forward_sum_1/num_of_samples

      # left shift the parameters
            shifted[i, j] -= np.pi

      # parameter-shift for the second term of both the terms of the objective function

      # backward evaluation
            backward_sum_1 = 0
            result = measure_rho(shifted)
            for sample in range(num_of_samples):
                sample_result_array = np.array([result[i][sample] for i in range(int(num_wires))])
                nn_result = neural_fn(torch.from_numpy(sample_result_array))
                backward_sum_1 += (nn_result[0].detach().numpy())**((alpha-1)/alpha)

      # normalize the backward sum
            backward_sum_1 = backward_sum_1/num_of_samples

      # parameter-shift for the second term
            shifted = param_init.copy()
    
      # right shift the parameters
            shifted[i, j] += np.pi/2

      # forward evaluation
            forward_sum_2 = 0
            result = measure_sigma(shifted)
            for sample in range(num_of_samples):
                sample_result_array = np.array([result[i][sample] for i in range(int(num_wires))])
                nn_result = neural_fn(torch.from_numpy(sample_result_array))
                forward_sum_2 += nn_result[0].detach().numpy()

      # normalize this sum
            forward_sum_2 = forward_sum_2/num_of_samples

      # left shift the parameters
            shifted[i, j] -= np.pi

      # parameter-shift for the second term of both the terms of the objective function

      # backward evaluation
            backward_sum_2 = 0
            result = measure_sigma(shifted)
            for sample in range(num_of_samples):
                sample_result_array = np.array([result[i][sample] for i in range(int(num_wires))])
                nn_result = neural_fn(torch.from_numpy(sample_result_array))
                backward_sum_2 += nn_result[0].detach().numpy()

      # normalize the backward sum
            backward_sum_2 = backward_sum_2/num_of_samples

      # parameter-shift rule
            gradients[i, j] = 0.5*alpha * (forward_sum_1 - backward_sum_1) + (0.5*(1-alpha) * (forward_sum_2 - backward_sum_2))

  # first copy the quantum circuit parameters before updating it
    prev_param_init = param_init.copy()

  # update the quantum circuit parameters
    param_init += learning_rate*gradients

  # evaluate the gradient with respect to the neural network parameters

    # evaluate the first term
    grad_w1 = []
    grad_b1 = []
    for layer_index in range(len(hidden_layer)):
        grad_w1.append(torch.zeros_like(neural_fn.hidden_layer_modules[layer_index].weight))
        grad_b1.append(torch.zeros_like(neural_fn.hidden_layer_modules[layer_index].bias))
    grad_w2 = torch.zeros_like(neural_fn.lin_end.weight)
    grad_b2 = torch.zeros_like(neural_fn.lin_end.bias)

    result = measure_rho(prev_param_init)
    for sample in range(num_of_samples):
        sample_result_array = np.array([result[i][sample] for i in range(int(num_wires))])
        nn_result = neural_fn(torch.from_numpy(sample_result_array))
        nn_result.backward()
        for layer_index in range(len(hidden_layer)):
            grad_w1[layer_index] += ((nn_result[0].detach().numpy())**((-1)/alpha))*neural_fn.hidden_layer_modules[layer_index].weight.grad*(1/num_of_samples)
            grad_b1[layer_index] += ((nn_result[0].detach().numpy())**((-1)/alpha))*neural_fn.hidden_layer_modules[layer_index].bias.grad*(1/num_of_samples)
        grad_w2 += ((nn_result[0].detach().numpy())**((-1)/alpha))*neural_fn.lin_end.weight.grad*(1/num_of_samples)
        grad_b2 += ((nn_result[0].detach().numpy())**((-1)/alpha))*neural_fn.lin_end.bias.grad*(1/num_of_samples)
        for layer_index in range(len(hidden_layer)):
            neural_fn.hidden_layer_modules[layer_index].weight.grad.zero_()
            neural_fn.hidden_layer_modules[layer_index].bias.grad.zero_()
        neural_fn.lin_end.weight.grad.zero_()
        neural_fn.lin_end.bias.grad.zero_()


  # evaluate the second term
    grad_w1_2 = []
    grad_b1_2 = []
    for layer_index in range(len(hidden_layer)):
        grad_w1_2.append(torch.zeros_like(neural_fn.hidden_layer_modules[layer_index].weight))
        grad_b1_2.append(torch.zeros_like(neural_fn.hidden_layer_modules[layer_index].bias))
    grad_w2_2 = torch.zeros_like(neural_fn.lin_end.weight.grad)
    grad_b2_2 = torch.zeros_like(neural_fn.lin_end.bias.grad)

    result = measure_sigma(prev_param_init)
    for sample in range(num_of_samples):
        sample_result_array = np.array([result[i][sample] for i in range(int(num_wires))])
        nn_result = neural_fn(torch.from_numpy(sample_result_array))
        nn_result.backward()
        for layer_index in range(len(hidden_layer)):
            grad_w1_2[layer_index] += (nn_result[0].detach().numpy()**(1))*neural_fn.hidden_layer_modules[layer_index].weight.grad*(1/num_of_samples)
            grad_b1_2[layer_index] += (nn_result[0].detach().numpy()**(1))*neural_fn.hidden_layer_modules[layer_index].bias.grad*(1/num_of_samples)
        grad_w2_2 += (nn_result[0].detach().numpy()**(1))*neural_fn.lin_end.weight.grad*(1/num_of_samples)
        grad_b2_2 += (nn_result[0].detach().numpy()**(1))*neural_fn.lin_end.bias.grad*(1/num_of_samples)
        for layer_index in range(len(hidden_layer)):
            neural_fn.hidden_layer_modules[layer_index].weight.grad.zero_()
            neural_fn.hidden_layer_modules[layer_index].bias.grad.zero_()
        neural_fn.lin_end.weight.grad.zero_()
        neural_fn.lin_end.bias.grad.zero_()

  # evaluate the difference, i.e., the gradient
    nn_grad_W1 = []
    nn_grad_b1 = []
    for layer_index in range(len(hidden_layer)):
        nn_grad_W1.append(grad_w1[layer_index] - grad_w1_2[layer_index])
        nn_grad_b1.append(grad_b1[layer_index] - grad_b1_2[layer_index])
    nn_grad_W2 = grad_w2 - grad_w2_2
    nn_grad_b2 = grad_b2 - grad_b2_2

  # update the NN weights and normalize them
    with torch.no_grad():
        for layer_index in range(len(hidden_layer)):
            neural_fn.hidden_layer_modules[layer_index].weight += learning_rate*(alpha-1)*nn_grad_W1[layer_index]
            neural_fn.hidden_layer_modules[layer_index].bias += learning_rate*(alpha-1)*nn_grad_b1[layer_index]
        neural_fn.lin_end.weight += learning_rate*(alpha-1)*nn_grad_W2
        neural_fn.lin_end.bias += learning_rate*(alpha-1)*nn_grad_b2

  # evaluate the cost function at these parameters
    first_term = 0
    result = measure_rho(param_init)
    for sample in range(num_of_samples):
        sample_result_array = np.array([result[i][sample] for i in range(int(num_wires))])
        nn_result = neural_fn(torch.from_numpy(sample_result_array))
        first_term += (nn_result[0].detach().numpy())**((alpha-1)/alpha)

  # normalize the cost sum
    first_term = first_term/num_of_samples

  # # Second term evaluation
    second_term = 0
    result = measure_sigma(param_init)
    for sample in range(num_of_samples):
        sample_result_array = np.array([result[i][sample] for i in range(int(num_wires))])
        nn_result = neural_fn(torch.from_numpy(sample_result_array))
        second_term += nn_result[0].detach().numpy()

  # normalize the second term sum
    second_term = second_term/num_of_samples

    # add the cost function to the store
    cost_func_store.append((1/(alpha-1))*np.log(alpha*first_term + (1-alpha)*second_term))

  # print the cost
    print((1/(alpha-1))*np.log(alpha*first_term + (1-alpha)*second_term))

In [ ]:
#@title Optimization using Gradient Descent with neural network
# This also works for small alpha

# parameters of the optimization
num_of_epochs = 2500
learning_rate = 0.08
num_of_samples = 100
dimension = num_wires
hidden_layer = [2]
alpha = 1.05


# initialize the neural network and quantum circuit parameters
neural_fn = neural_function(dimension, hidden_layer)
param_init = np.random.random(qml.RandomLayers.shape(n_layers=num_layers, n_rotations=3))


# intialize the cost function store
cost_func_store = []


# start the training
for epoch in range(1, num_of_epochs):


  # evaluate the gradient with respect to the quantum circuit parameters
    gradients = np.zeros_like((param_init))
    for i in range(len(gradients)):
        for j in range(len(gradients[0])):

      # copy the parameters
            shifted = param_init.copy()

      # right shift the parameters
            shifted[i, j] += np.pi/2

      # parameter-shift for the first term

      # forward evaluation
            forward_sum_1 = 0
            result = measure_rho(shifted)
            for sample in range(num_of_samples):
                sample_result_array = np.array([result[i][sample] for i in range(int(num_wires))])
                nn_result = neural_fn(torch.from_numpy(sample_result_array))
                forward_sum_1 += np.exp(((alpha-1)/alpha)*nn_result[0].detach().numpy())

      # normalize this sum
            forward_sum_1 = forward_sum_1/num_of_samples

      # left shift the parameters
            shifted[i, j] -= np.pi

      # parameter-shift for the second term of both the terms of the objective function

      # backward evaluation
            backward_sum_1 = 0
            result = measure_rho(shifted)
            for sample in range(num_of_samples):
                sample_result_array = np.array([result[i][sample] for i in range(int(num_wires))])
                nn_result = neural_fn(torch.from_numpy(sample_result_array))
                backward_sum_1 += np.exp(((alpha-1)/alpha)*nn_result[0].detach().numpy())

      # normalize the backward sum
            backward_sum_1 = backward_sum_1/num_of_samples

      # parameter-shift for the second term
            shifted = param_init.copy()
        
      # right shift the parameters
            shifted[i, j] += np.pi/2

      # forward evaluation
            forward_sum_2 = 0
            result = measure_sigma(shifted)
            for sample in range(num_of_samples):
                sample_result_array = np.array([result[i][sample] for i in range(int(num_wires))])
                nn_result = neural_fn(torch.from_numpy(sample_result_array))
                forward_sum_2 += np.exp(nn_result[0].detach().numpy())

      # normalize this sum
            forward_sum_2 = forward_sum_2/num_of_samples

      # left shift the parameters
            shifted[i, j] -= np.pi

      # parameter-shift for the second term of both the terms of the objective function

      # backward evaluation
            backward_sum_2 = 0
            result = measure_sigma(shifted)
            for sample in range(num_of_samples):
                sample_result_array = np.array([result[i][sample] for i in range(int(num_wires))])
                nn_result = neural_fn(torch.from_numpy(sample_result_array))
                backward_sum_2 += np.exp(nn_result[0].detach().numpy())

      # normalize the backward sum
            backward_sum_2 = backward_sum_2/num_of_samples

      # parameter-shift rule
            gradients[i, j] = 0.5*alpha * (forward_sum_1 - backward_sum_1) + (0.5*(1-alpha) * (forward_sum_2 - backward_sum_2))

  # first copy the quantum circuit parameters before updating it
    prev_param_init = param_init.copy()

  # update the quantum circuit parameters
    param_init += learning_rate*gradients

  # evaluate the gradient with respect to the neural network parameters

    # evaluate the first term
    grad_w1 = []
    grad_b1 = []
    for layer_index in range(len(hidden_layer)):
        grad_w1.append(torch.zeros_like(neural_fn.hidden_layer_modules[layer_index].weight))
        grad_b1.append(torch.zeros_like(neural_fn.hidden_layer_modules[layer_index].bias))
    grad_w2 = torch.zeros_like(neural_fn.lin_end.weight)
    grad_b2 = torch.zeros_like(neural_fn.lin_end.bias)

    result = measure_rho(prev_param_init)
    for sample in range(num_of_samples):
        sample_result_array = np.array([result[i][sample] for i in range(int(num_wires))])
        nn_result = neural_fn(torch.from_numpy(sample_result_array))
        nn_result.backward()
        for layer_index in range(len(hidden_layer)):
            grad_w1[layer_index] += (np.exp(((alpha-1)/alpha)*nn_result[0].detach().numpy()))*neural_fn.hidden_layer_modules[layer_index].weight.grad*(1/num_of_samples)
            grad_b1[layer_index] += (np.exp(((alpha-1)/alpha)*nn_result[0].detach().numpy()))*neural_fn.hidden_layer_modules[layer_index].bias.grad*(1/num_of_samples)
        grad_w2 += (np.exp(((alpha-1)/alpha)*nn_result[0].detach().numpy()))*neural_fn.lin_end.weight.grad*(1/num_of_samples)
        grad_b2 += (np.exp(((alpha-1)/alpha)*nn_result[0].detach().numpy()))*neural_fn.lin_end.bias.grad*(1/num_of_samples)
        for layer_index in range(len(hidden_layer)):
            neural_fn.hidden_layer_modules[layer_index].weight.grad.zero_()
            neural_fn.hidden_layer_modules[layer_index].bias.grad.zero_()
        neural_fn.lin_end.weight.grad.zero_()
        neural_fn.lin_end.bias.grad.zero_()


  # evaluate the second term
    grad_w1_2 = []
    grad_b1_2 = []
    for layer_index in range(len(hidden_layer)):
        grad_w1_2.append(torch.zeros_like(neural_fn.hidden_layer_modules[layer_index].weight))
        grad_b1_2.append(torch.zeros_like(neural_fn.hidden_layer_modules[layer_index].bias))
    grad_w2_2 = torch.zeros_like(neural_fn.lin_end.weight.grad)
    grad_b2_2 = torch.zeros_like(neural_fn.lin_end.bias.grad)

    result = measure_sigma(prev_param_init)
    for sample in range(num_of_samples):
        sample_result_array = np.array([result[i][sample] for i in range(int(num_wires))])
        nn_result = neural_fn(torch.from_numpy(sample_result_array))
        nn_result.backward()
        for layer_index in range(len(hidden_layer)):
            grad_w1_2[layer_index] += (np.exp(nn_result[0].detach().numpy()))*neural_fn.hidden_layer_modules[layer_index].weight.grad*(1/num_of_samples)
            grad_b1_2[layer_index] += (np.exp(nn_result[0].detach().numpy()))*neural_fn.hidden_layer_modules[layer_index].bias.grad*(1/num_of_samples)
        grad_w2_2 += (np.exp(nn_result[0].detach().numpy()))*neural_fn.lin_end.weight.grad*(1/num_of_samples)
        grad_b2_2 += (np.exp(nn_result[0].detach().numpy()))*neural_fn.lin_end.bias.grad*(1/num_of_samples)
        for layer_index in range(len(hidden_layer)):
            neural_fn.hidden_layer_modules[layer_index].weight.grad.zero_()
            neural_fn.hidden_layer_modules[layer_index].bias.grad.zero_()
        neural_fn.lin_end.weight.grad.zero_()
        neural_fn.lin_end.bias.grad.zero_()

  # evaluate the difference, i.e., the gradient
    nn_grad_W1 = []
    nn_grad_b1 = []
    for layer_index in range(len(hidden_layer)):
        nn_grad_W1.append(grad_w1[layer_index] - grad_w1_2[layer_index])
        nn_grad_b1.append(grad_b1[layer_index] - grad_b1_2[layer_index])
    nn_grad_W2 = grad_w2 - grad_w2_2
    nn_grad_b2 = grad_b2 - grad_b2_2

  # update the NN weights and normalize them
    with torch.no_grad():
        for layer_index in range(len(hidden_layer)):
            neural_fn.hidden_layer_modules[layer_index].weight += learning_rate*(alpha-1)*nn_grad_W1[layer_index]
            neural_fn.hidden_layer_modules[layer_index].bias += learning_rate*(alpha-1)*nn_grad_b1[layer_index]
        neural_fn.lin_end.weight += learning_rate*(alpha-1)*nn_grad_W2
        neural_fn.lin_end.bias += learning_rate*(alpha-1)*nn_grad_b2

  # evaluate the cost function at these parameters
    first_term = 0
    result = measure_rho(param_init)
    for sample in range(num_of_samples):
        sample_result_array = np.array([result[i][sample] for i in range(int(num_wires))])
        nn_result = neural_fn(torch.from_numpy(sample_result_array))
        first_term += np.exp(((alpha-1)/alpha)*nn_result[0].detach().numpy())

  # normalize the cost sum
    first_term = first_term/num_of_samples

  # # Second term evaluation
    second_term = 0
    result = measure_sigma(param_init)
    for sample in range(num_of_samples):
        sample_result_array = np.array([result[i][sample] for i in range(int(num_wires))])
        nn_result = neural_fn(torch.from_numpy(sample_result_array))
        second_term += np.exp(nn_result[0].detach().numpy())

  # normalize the second term sum
    second_term = second_term/num_of_samples

    # add the cost function to the store
    cost_func_store.append((1/(alpha-1))*np.log(alpha*first_term + (1-alpha)*second_term))

  # print the cost
    print((1/(alpha-1))*np.log(alpha*first_term + (1-alpha)*second_term))

In [ ]:
def binary_array_to_decimal(arr):
    arr = arr.astype(int)
    # Replace -1 with 0
    arr = np.where(arr == -1, 0, arr)
    # Convert the array to a binary string
    binary_string = ''.join(str(i) for i in arr)
    # Convert the binary string to a decimal value
    decimal_value = int(binary_string, 2)
    return decimal_value

In [ ]:
#@title Optimization using Gradient Descent without neural network.

# parameters of the optimization
num_of_epochs = 2500
learning_rate = 0.8
num_of_samples = 100
deviation = 1
alpha = 1.05
seed_no = 42

# we store the eigenvalues in a array
W = deviation*np.random.rand(64)
param_init = np.random.random(qml.RandomLayers.shape(n_layers=num_layers, n_rotations=3))


# intialize the cost function store
cost_func_store = []


# start the training
for epoch in range(1, num_of_epochs):


  # evaluate the gradient with respect to the quantum circuit parameters
    gradients = np.zeros_like((param_init))
    for i in range(len(gradients)):
        for j in range(len(gradients[0])):

      # copy the parameters
            shifted = param_init.copy()

      # right shift the parameters
            shifted[i, j] += np.pi/2

      # parameter-shift for the first term

      # forward evaluation
            forward_sum_1 = 0
            result = measure_rho(shifted)
            for sample in range(num_of_samples):
                sample_result_array = np.array([result[i][sample] for i in range(int(num_wires))])
                forward_sum_1 += np.exp(((alpha-1)/alpha)*W[binary_array_to_decimal(sample_result_array)])

      # normalize this sum
            forward_sum_1 = forward_sum_1/num_of_samples

      # left shift the parameters
            shifted[i, j] -= np.pi

      # parameter-shift for the second term of both the terms of the objective function

      # backward evaluation
            backward_sum_1 = 0
            result = measure_rho(shifted)
            for sample in range(num_of_samples):
                sample_result_array = np.array([result[i][sample] for i in range(int(num_wires))])
                backward_sum_1 += np.exp(((alpha-1)/alpha)*W[binary_array_to_decimal(sample_result_array)])

      # normalize the backward sum
            backward_sum_1 = backward_sum_1/num_of_samples

      # parameter-shift for the second term
            shifted = param_init.copy()
    
      # right shift the parameters
            shifted[i, j] += np.pi/2

      # forward evaluation
            forward_sum_2 = 0
            result = measure_sigma(shifted)
            for sample in range(num_of_samples):
                sample_result_array = np.array([result[i][sample] for i in range(int(num_wires))])
                forward_sum_2 += np.exp(W[binary_array_to_decimal(sample_result_array)])

      # normalize this sum
            forward_sum_2 = forward_sum_2/num_of_samples

      # left shift the parameters
            shifted[i, j] -= np.pi

      # parameter-shift for the second term of both the terms of the objective function

      # backward evaluation
            backward_sum_2 = 0
            result = measure_sigma(shifted)
            for sample in range(num_of_samples):
                sample_result_array = np.array([result[i][sample] for i in range(int(num_wires))])
                backward_sum_2 += np.exp(W[binary_array_to_decimal(sample_result_array)])

      # normalize the backward sum
            backward_sum_2 = backward_sum_2/num_of_samples

      # parameter-shift rule
            gradients[i, j] = 0.5*alpha * (forward_sum_1 - backward_sum_1) + (0.5*(1-alpha) * (forward_sum_2 - backward_sum_2))

  # first copy the quantum circuit parameters before updating it
    prev_param_init = param_init.copy()

  # update the quantum circuit parameters
    param_init += learning_rate*gradients

    # evaluate the gradient with respect to the eigenvalues
    
    dW = np.zeros(64)
    result_rho = measure_rho(prev_param_init)
    result_sigma = measure_sigma(prev_param_init)
    for i in range(64):
        E = np.zeros(64)
        E[i] = 1
        for sample in range(num_of_samples):
            sample_result_array_rho = np.array([result_rho[i][sample] for i in range(int(num_wires))])
            sample_result_array_sigma = np.array([result_sigma[i][sample] for i in range(int(num_wires))])
            dW_first_term = E[binary_array_to_decimal(sample_result_array_rho)]*np.exp(((alpha-1)/alpha)*W[i])
            dW_sec_term = E[binary_array_to_decimal(sample_result_array_sigma)]*np.exp(W[i])
            dW[i] += dW_first_term - dW_sec_term
                        
        dW[i] = dW[i]/num_of_samples
        
    W += learning_rate*(alpha-1)*dW



  # evaluate the cost function at these parameters
    first_term = 0
    result = measure_rho(param_init)
    for sample in range(num_of_samples):
        sample_result_array = np.array([result[i][sample] for i in range(int(num_wires))])
        first_term += np.exp(((alpha-1)/alpha)*W[binary_array_to_decimal(sample_result_array)])

  # normalize the cost sum
    first_term = first_term/num_of_samples

  # # Second term evaluation
    second_term = 0
    result = measure_sigma(param_init)
    for sample in range(num_of_samples):
        sample_result_array = np.array([result[i][sample] for i in range(int(num_wires))])
        second_term += np.exp(W[binary_array_to_decimal(sample_result_array)])

  # normalize the second term sum
    second_term = second_term/num_of_samples

    # add the cost function to the store
    cost_func_store.append((1/(alpha-1))*np.log(alpha*first_term + (1-alpha)*second_term))

  # print the cost
    print((1/(alpha-1))*np.log(alpha*first_term + (1-alpha)*second_term))

In [ ]:
print(seed_no)